# Retriever and Chain with Langchain

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("attention3.pdf")
docs=loader.load()
docs

[Document(metadata={'source': 'attention3.pdf', 'page': 0}, page_content='Self-attention, sometimes called intra-attention is an attention mechanism relating \ndifferent positions of a single sequence in order to compute a representation of the \nsequence. Self-attention has been used successfully in a variety of tasks including reading \ncomprehension, abstractive summarization, textual entailment and learning task-\nindependent sentence representations ')]

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunk_documents=text_splitter.split_documents(docs)
chunk_documents

[Document(metadata={'source': 'attention3.pdf', 'page': 0}, page_content='Self-attention, sometimes called intra-attention is an attention mechanism relating \ndifferent positions of a single sequence in order to compute a representation of the \nsequence. Self-attention has been used successfully in a variety of tasks including reading \ncomprehension, abstractive summarization, textual entailment and learning task-\nindependent sentence representations')]

In [4]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

llm = Ollama(model="llama3.2")
embeddings = OllamaEmbeddings(model="llama3.2")

db=FAISS.from_documents(chunk_documents, embeddings)
db

C:\Users\arsal\AppData\Local\Temp\ipykernel_11256\2217773708.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [5]:
query='Self-attention, sometimes called intra-attention is an attention mechanism relating '
retrieved_result=db.similarity_search(query)
print(retrieved_result[0].page_content)

Self-attention, sometimes called intra-attention is an attention mechanism relating 
different positions of a single sequence in order to compute a representation of the 
sequence. Self-attention has been used successfully in a variety of tasks including reading 
comprehension, abstractive summarization, textual entailment and learning task-
independent sentence representations


In [8]:
# Design Chat prompt template
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""
Answer the following questions based only on the provided context.
<context>
{context}
</context>
                                        
Question:{input}
""")

In [9]:
# chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following questions based only on the provided context.\n<context>\n{context}\n</context>\n                                        \nQuestion:{input}\n'), additional_kwargs={})])
| Ollama(model='llama3.2')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [10]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002137E61DC30>, search_kwargs={})

In [11]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [14]:
response=retrieval_chain.invoke({"input":"intra-attention"})
response['answer']

'Intra-attention refers to self-attention, also known as intra-attention.'